## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append('../')
from config import g_key
sys.path.pop()

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.07,172.79,82.13,76,44,9.93,light rain
1,1,Souillac,MU,-20.52,57.52,75.58,69,75,12.66,broken clouds
2,2,Georgetown,MY,5.41,100.34,83.89,83,20,3.44,few clouds
3,3,Leningradskiy,RU,69.38,178.42,34.25,94,97,4.68,overcast clouds
4,4,Rikitea,PF,-23.12,-134.97,76.10,78,99,12.53,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= min_temp)
    & (city_data_df['Max Temp'] <= max_temp)
]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             248
City                   248
Country                248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,82.13,light rain,3.07,172.79,
1,Souillac,MU,75.58,broken clouds,-20.52,57.52,
2,Georgetown,MY,83.89,few clouds,5.41,100.34,
4,Rikitea,PF,76.10,overcast clouds,-23.12,-134.97,
5,Morondava,MG,75.36,scattered clouds,-20.28,44.28,
10,Atuona,PF,78.80,few clouds,-9.80,-139.03,
15,Mahebourg,MU,75.49,broken clouds,-20.41,57.70,
18,Bambous Virieux,MU,75.45,broken clouds,-20.34,57.76,
20,Vilhena,BR,79.02,clear sky,-12.74,-60.15,
24,Purna,IN,78.91,broken clouds,19.18,77.05,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, hotel in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = hotel['Lat']
    lng = hotel['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_Vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = '''
<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City Name</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
'''

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))